# GMC - Homogeneous Catalysis

Simple example of cobalt-catalyzed hydroformylation reactions adapted from an accepted mechanism predicted through [molecular graph and reaction network analysis](https://pubs.rsc.org/en/content/articlehtml/2017/sc/c7sc03628k).

In [1]:
import sqlite3
import numpy as np

In [4]:
## CREATE REACTION NETWORK DATABASE ##

create_metadata_table = """
    CREATE TABLE metadata (
            number_of_species   INTEGER NOT NULL,
            number_of_reactions INTEGER NOT NULL
    );
"""

insert_metadata = """
    INSERT INTO metadata VALUES (?, ?)
"""

# It is important that reaction_id is the primary key
# otherwise the network loader will be extremely slow.

create_reactions_table = """
    CREATE TABLE reactions (
            reaction_id                     INTEGER NOT NULL PRIMARY KEY,
            number_of_reactants             INTEGER NOT NULL,
            number_of_products              INTEGER NOT NULL,
            reactant_1                      INTEGER NOT NULL,
            reactant_2                      INTEGER NOT NULL,
            product_1                       INTEGER NOT NULL,
            product_2                       INTEGER NOT NULL,
            rate                            REAL NOT NULL
    );
"""


insert_reaction = """
    INSERT INTO reactions VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

In [5]:
def create_reaction_sqlite(path, stmts, num_species, reactions):
    ''' Create an empty sqlite, name, with a table for the metadata and 
    reactions'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()
    
    for stmt in stmts:    
        rn_cur.execute(stmt)
    rn_con.commit()
    
    rn_cur.execute(
        insert_metadata,
        (num_species, np.shape(reactions)[0]))

    rn_con.commit()
    
    for reaction in reactions:
        rn_cur.execute(insert_reaction, tuple(reaction))
    rn_con.commit()
    
    rn_con.close()

In [6]:
## CREATE INITIAL STATE DATABSE ##

create_initial_state_table = """
    CREATE TABLE initial_state (
            species_id             INTEGER NOT NULL PRIMARY KEY,
            count                  INTEGER NOT NULL
    );
"""

create_trajectories_table = """
    CREATE TABLE trajectories (
            seed                INTEGER NOT NULL,
            step                INTEGER NOT NULL,
            time                REAL NOT NULL,
            reaction_id         INTEGER NOT NULL
    );
"""

create_factors_table = """
    CREATE TABLE factors (
            factor_zero      REAL NOT NULL,
            factor_two       REAL NOT NULL,
            factor_duplicate REAL NOT NULL
    );
"""

create_interrupt_state = """
    CREATE TABLE interrupt_state (
            seed                    INTEGER NOT NULL,
            species_id              INTEGER NOT NULL,
            count                   INTEGER NOT NULL      
    );
"""

create_interrupt_cutoff = """
    CREATE TABLE interrupt_cutoff (
            seed                    INTEGER NOT NULL,
            step                    INTEGER NOT NULL,
            time                    INTEGER NOT NULL
    );
"""

In [7]:
def create_state_sqlite(path, stmts, species, initial_state,
                        factor_zero = 1, factor_two = 1, 
                        factor_duplicate = 0.5):
    
    '''Create state sqlite which consists of tables: 
    initial state, trajectories, factors, interrupt_state 
    and interrupt_cutoff'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()
    
    for stmt in stmts:    
        rn_cur.execute(stmt)
    rn_con.commit()
    
    insert_into_factors = "INSERT INTO factors VALUES (?,?,?)"

    rn_cur.execute(
        insert_into_factors,
        (factor_zero, factor_two, factor_duplicate)
    )

    for i in range(len(species)):
        rn_cur.execute(
            "INSERT INTO initial_state VALUES (?,?)",
            (i, initial_state.get(i,0)))

    rn_con.commit()
    rn_con.close()

In [8]:
## MAPPING OF SPECIES ##
species_thermo = {'3': {'g': 7.5}, '3_p': {'g': -0.7}, 'R': {'g': 0.0}, 
                  '2': {'g': -11.7}, 'I_1':{'g': -3.6}, 'I_2':{'g': -11.7},
                  'I_3':{'g': -21.4}, '27_p':{'g': -5.6}, 'P_side':{'g': -23.8},
                  '14':{'g': -16.2}, '121':{'g': -15.0}, 'I_4':{'g': -17.3},
                  '122':{'g': 0.8}, 'I_4-3':{'g': -13.9}, '20':{'g': 13.2}, 
                  '109':{'g': 19.2}, 'I_5_p':{'g': -11.7}, 'I_5':{'g': -4.6}, 
                  'P_p':{'g': -16.9}, 'P':{'g': -21.0},'P_pp':{'g': -7.9}, 
                  'I_5-2':{'g': 0.0}, 'I_5-2_p':{'g': -6.6}, 'I_4-2':{'g': -18.7}, 
                  '5':{'g': 26.4}, '23':{'g': 22.1}, '7':{'g': 5.8}, '24':{'g': 20.7}, 
                  '99':{'g': 36.9}, '24_p': {'g': 8.1}, '71': {'g': 6.6}}

In [9]:
## TRANSITION STATES ##
transition_states = {'T_9': {'g': 7.5}, 'T_1': {'g': -1.7}, 'T_7': {'g': -1.7},
                     'T_2': {'g': -0.5}, 'T_8': {'g': 5.4}, 'T_3': {'g': -11.6},
                     'T_4': {'g': -8.2}, 'T_5': {'g': -5.4}, 'T_6': {'g': -2.8},
                     'T_6-2': {'g': 1.4}, 'T_5-2': {'g': 1.3}, 'T_4-2': {'g': -18.7},
                     'T_14': {'g': 62.1}, 'T_12': {'g': 62.8}, 'T_11': {'g': 10.1}, 
                     'T_10': {'g': 18.2}, 'T_13': {'g': 96.1}, 'T_15': {'g': 32.9},
                     'T_16': {'g': 77.4}, 'T_17': {'g': 85.0}, 'T_18': {'g': 34.2},
                     'T_19': {'g': 23.0}, 'T_20': {'g': 89.4}
                    }

In [10]:
mapping = {m: i for i, m in enumerate(species_thermo, 0)}

In [11]:
## CREATE INITIAL STATE DATABSE ##

initial_state = {3: 30000, 14: 30000, 99: 30000}

stmts = [create_initial_state_table, create_trajectories_table, 
         create_factors_table, create_interrupt_cutoff, 
         create_interrupt_state]

create_state_sqlite("initial_state.sqlite", stmts, species_thermo, initial_state)

In [12]:
# For simplicity, each reaction is treated as 
# having a single reactant and product.
# First reactions are listed as 
# ['reactant', 'product', 'transition state']
# the rates are then calculated with the Eyring equation
# then the reactions are formated for the .sqlite table
# each reaction can run in both directions

reactions_base = [
    ['3', '3_p', 'T_9'],
    ['3_p', 'R', None],
    ['R', 'I_1', None],
    ['2', 'R', None],
    ['I_1', 'I_2', 'T_1'],
    ['27_p', 'I_2', 'T_7'],
    ['P_side', '27_p', 'T_8'],
    ['I_2', 'I_3', 'T_2'],
    ['I_3', 'I_4', 'T_3'],
    ['I_4', 'I_5_p', 'T_4'],
    ['I_5_p', 'I_5', 'T_5'],
    ['I_5', 'P_p', 'T_6'],
    ['P_p', 'P', None],
    ['P', 'P_pp', None],
    ['P_pp', 'I_5-2', 'T_6-2'],
    ['I_5-2', 'I_5-2_p', 'T_5-2'],
    ['I_5-2_p', 'I_4-2', 'T_4-2'],
    ['14', '121', 'T_14'],
    ['121', 'I_4', 'T_12'],
    ['122', 'I_4', 'T_11'],
    ['I_4-3', 'I_4', None],
    ['20', 'I_4-3', 'T_10'],
    ['109', '20', 'T_13'],
    ['99', '24', 'T_20'],
    ['24', '24_p', 'T_19'],
    ['7', '24_p', None],
    ['24_p', '23', 'T_18'],
    ['5', '23', 'T_16'],
    ['I_3', '5', 'T_15'],
    ['71', '23', 'T_17']
    
]

In [13]:
## Calculate rates with Eyring equation 

# Room temperature (25 C) in Kelvin
ROOM_TEMP = 298.15

def eyring(dg_barrier, kappa=1.0, temperature=ROOM_TEMP):

    # Boltzmann constant in eV / K
    KB = 8.617333262 * 10 ** -5

    # Planck constant in eV * s
    PLANCK = 4.135667696 * 10 ** -15

    if dg_barrier <= 0:
        return kappa * KB * temperature / PLANCK
    else:
        return kappa * KB * temperature / PLANCK * np.exp(-dg_barrier / (KB * temperature))


In [14]:
## Format for .sqlite file ##
reactions = []
num_reactions = 0

for react in reactions_base:
    
    if(react[2] != None):
        # convert from kcal mol-1 to eV
        rate_forward = eyring(0.0434 * (transition_states[react[2]]['g'] 
                                - species_thermo[react[0]]['g']))
    else:
        rate_forward = 1e12
    
    # always 1 reactant and 1 product
    reactions.append([num_reactions, 1, 1, mapping[react[0]], -1, 
                      mapping[react[1]], -1, rate_forward])
    num_reactions = num_reactions + 1
    
    if(react[2] != None):
        # convert from kcal mol-1 to eV
        rate_backwards = eyring(0.0434 * (transition_states[react[2]]['g'] 
                                - species_thermo[react[1]]['g']))
    else:
        rate_backwards = 1e12
    
    # always 1 reactant and 1 product
    reactions.append([num_reactions, 1, 1, mapping[react[1]], -1, 
                      mapping[react[0]], -1, rate_backwards])
    num_reactions = num_reactions + 1
    

In [15]:
## CREATE REACTION NETWORK DATABSE ##

stmts = [create_metadata_table, create_reactions_table]
create_reaction_sqlite("rn.sqlite", stmts, len(species_thermo), reactions)
